The code for this is found: https://sdmx1.readthedocs.io/en/v2.22.0/sources.html

# API

In [27]:
import sdmx

client = sdmx.Client()
url = (
    # Base URL
    "http://dataservices.imf.org/REST/SDMX_XML.svc/CompactData/"
    # Data flow ID and key
    "DOT/M..TMG_CIF_USD.US+CN+B0"
    # Query parameters, including format
    "?startPeriod=2000&format=sdmx-2.1"
)

# Retrieve an SDMX-ML 2.1 data message
message = client.get(url=url)

# Convert the single data set to pandas.Series with multi-index
df = sdmx.to_pandas(message.data[0])

xml.Reader got no structure=… argument for StructureSpecificTimeSeriesData


In [28]:
df.head()

FREQ  REF_AREA  INDICATOR    COUNTERPART_AREA  UNIT_MULT  TIME_FORMAT  TIME_PERIOD
M     EC        TMG_CIF_USD  B0                6          P1M          2000-01        21.124353
                                                                       2000-02        25.516058
                                                                       2000-03        24.095966
                                                                       2000-04        23.262880
                                                                       2000-05        30.677035
Name: value, dtype: float64

# Data

In [29]:
# Assuming 'df' is your existing Series with MultiIndex
# Convert to DataFrame and reset the index (keeps original data intact)
df_flat = df.reset_index()
df_flat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197502 entries, 0 to 197501
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   FREQ              197502 non-null  object 
 1   REF_AREA          197502 non-null  object 
 2   INDICATOR         197502 non-null  object 
 3   COUNTERPART_AREA  197502 non-null  object 
 4   UNIT_MULT         197502 non-null  object 
 5   TIME_FORMAT       197502 non-null  object 
 6   TIME_PERIOD       197502 non-null  object 
 7   value             197502 non-null  float64
dtypes: float64(1), object(7)
memory usage: 12.1+ MB


In [30]:
import pandas as pd

# Assuming your dataframe is named df_flat
df_wide = df_flat.pivot_table(
    index=['REF_AREA', 'TIME_PERIOD'],  # these will remain as index
    columns='COUNTERPART_AREA',  # each time period becomes a new column
    values='value',  # the values you want to spread across the columns
    aggfunc='first'  # assuming no duplicates in (REF_AREA, COUNTERPART_AREA, TIME_PERIOD)
)

# Reset column names and index if needed
df_wide.reset_index(inplace=True)
df_wide.columns.name = None  # Remove the columns' name

# Now df_wide is your wider dataframe where each time period is a new column


In [31]:

data = df_wide.rename(columns={
    'FREQ': 'Data Granularity',
    'REF_AREA': 'Importer_Code',
    'INDICATOR': 'Indicator',
    'COUNTERPART_AREA': 'Exporter_Code',

})


## Calculating Highest

In [32]:
def get_highest_source(row):
    total = row['B0'] + row['CN'] + row['US']
    max_value = max(row['B0'], row['CN'], row['US'])
    if max_value == row['B0']:
        return 'B0', max_value / total
    elif max_value == row['CN']:
        return 'CN', max_value / total
    else:
        return 'US', max_value / total

# Apply the function to create the new columns
data[['Highest_Source', 'Max_Share']] = data.apply(lambda row: pd.Series(get_highest_source(row)), axis=1)

# Display the result
data.head()


,Importer_Code,TIME_PERIOD,B0,CN,US,Highest_Source,Max_Share
0,1C_080,2000-01,3637.607053,457.369271,1646.845575,B0,0.633528
1,1C_080,2000-02,3904.274716,545.905916,1862.996468,B0,0.618433
2,1C_080,2000-03,4705.270448,646.915615,2206.118977,B0,0.622530
3,1C_080,2000-04,3638.390102,512.920319,1748.425258,B0,0.616704
4,1C_080,2000-05,4326.089665,617.827923,1756.750902,B0,0.645621


# Joining Geofile with IMF Stats

In [33]:
import geopandas as gpd

# Read the GeoJSON file
gdf = gpd.read_file('imports_app/geo/wab.geojson')

# Now you have a GeoDataFrame
print(gdf.crs)     # View coordinate reference system

EPSG:4326


In [34]:
# Drop the specified columns
columns_to_drop = ['iso3', 'status', 'color_code', 'french_short']
gdf = gdf.drop(columns=columns_to_drop, errors='ignore')  # errors='ignore' prevents error if column doesn't exist

gdf.dropna(subset=['iso_3166_1_alpha_2_codes'], inplace=True)

In [35]:
gdf.head()

,geo_point_2d,name,continent,region,iso_3166_1_alpha_2_codes,geometry
0,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,..."
1,"{ ""lon"": 63.169364370421164, ""lat"": 41.7504440...",Uzbekistan,Asia,Central Asia,UZ,"POLYGON ((70.97081 42.25467, 70.98054 42.26205..."
2,"{ ""lon"": -8.150578960214018, ""lat"": 53.1763816...",Ireland,Europe,Northern Europe,IE,"MULTIPOLYGON (((-9.97014 54.02083, -9.93833 53..."
3,"{ ""lon"": 38.841285734615539, ""lat"": 15.3732031...",Eritrea,Africa,Eastern Africa,ER,"MULTIPOLYGON (((40.13583 15.7525, 40.12861 15...."
5,"{ ""lon"": 8.234429172288662, ""lat"": 46.80256937...",Switzerland,Europe,Western Europe,CH,"POLYGON ((9.56672 47.54045, 9.5598 47.50209, 9..."


In [36]:
# Left join (keeps all rows from gdf_clean and matches with data)
merged = gdf.merge(
    data,
    left_on='iso_3166_1_alpha_2_codes',
    right_on='Importer_Code',
    how='left'
)

merged.head()

,geo_point_2d,name,continent,region,iso_3166_1_alpha_2_codes,geometry,Importer_Code,TIME_PERIOD,B0,CN,US,Highest_Source,Max_Share
0,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-01,8.485627,2.175394,2.759995,B0,0.632264
1,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-02,8.383154,2.214706,2.661651,B0,0.632237
2,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-03,6.614687,1.785681,2.040493,B0,0.633538
3,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-04,12.094331,3.327916,3.623060,B0,0.635029
4,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-05,7.864206,2.167408,2.272558,B0,0.639150


## Using API, get the names of the codes

In [37]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

url = "http://dataservices.imf.org/REST/SDMX_XML.svc/CodeList/CL_AREA_DOT"
response = requests.get(url)

# Parse the XML response
root = ET.fromstring(response.content)

# Define namespace mapping
namespaces = {
    'message': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message',
    'structure': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure'
}

# Extract codes and descriptions
area_codes = {}
for code in root.findall(".//structure:Code", namespaces):
    code_id = code.get("value")
    description = code.find("structure:Description", namespaces).text
    area_codes[code_id] = description

# Create a DataFrame for easier viewing
df_codes = pd.DataFrame([(k, v) for k, v in area_codes.items()], 
                       columns=['Code', 'Description'])

print(df_codes)

       Code                                        Description
0        AF                                        Afghanistan
1       F19                               Africa not allocated
2        AL                                            Albania
3        DZ                                            Algeria
4        AS                                     American Samoa
..      ...                                                ...
242  1C_440  Middle East, North Africa, Afghanistan, and Pa...
243     X88                       Other Countries n.i.e. (IMF)
244      F6                                 Sub-Saharan Africa
245     A10                                 Western Hemisphere
246     W00                    All Countries, excluding the IO

[247 rows x 2 columns]


In [38]:
# Convert df_codes DataFrame to a dictionary where keys are codes and values are descriptions
code_dict = dict(zip(df_codes['Code'], df_codes['Description']))

merged['Highest_Exporter'] = merged['Highest_Source'].map(code_dict)

merged.head()

,geo_point_2d,name,continent,region,iso_3166_1_alpha_2_codes,geometry,Importer_Code,TIME_PERIOD,B0,CN,US,Highest_Source,Max_Share,Highest_Exporter
0,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-01,8.485627,2.175394,2.759995,B0,0.632264,EU (Member States and Institutions of the Euro...
1,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-02,8.383154,2.214706,2.661651,B0,0.632237,EU (Member States and Institutions of the Euro...
2,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-03,6.614687,1.785681,2.040493,B0,0.633538,EU (Member States and Institutions of the Euro...
3,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-04,12.094331,3.327916,3.623060,B0,0.635029,EU (Member States and Institutions of the Euro...
4,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-05,7.864206,2.167408,2.272558,B0,0.639150,EU (Member States and Institutions of the Euro...


In [39]:
def bin(value):
    if value <= 0.25:
        return 1 
    elif value <= 0.5:
        return 2
    elif value <= 0.75:
        return 3
    else:
        return 4

merged['bin'] = merged['Max_Share'].apply(bin)

merged.head()
        

,geo_point_2d,name,continent,region,iso_3166_1_alpha_2_codes,geometry,Importer_Code,TIME_PERIOD,B0,CN,US,Highest_Source,Max_Share,Highest_Exporter,bin
0,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-01,8.485627,2.175394,2.759995,B0,0.632264,EU (Member States and Institutions of the Euro...,3
1,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-02,8.383154,2.214706,2.661651,B0,0.632237,EU (Member States and Institutions of the Euro...,3
2,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-03,6.614687,1.785681,2.040493,B0,0.633538,EU (Member States and Institutions of the Euro...,3
3,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-04,12.094331,3.327916,3.623060,B0,0.635029,EU (Member States and Institutions of the Euro...,3
4,"{ ""lon"": 32.386218272811753, ""lat"": 1.27996344...",Uganda,Africa,Eastern Africa,UG,"POLYGON ((33.9211 -1.00194, 33.92027 -1.00111,...",UG,2000-05,7.864206,2.167408,2.272558,B0,0.639150,EU (Member States and Institutions of the Euro...,3


## Visualizing

Static

In [40]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.colors as mcolors
from matplotlib.font_manager import FontProperties
from matplotlib.legend_handler import HandlerBase
import cartopy.crs as ccrs  # For map projections
import cartopy.feature as cfeature  # For map features

# Custom handler for touching legend patches
class TouchingRectanglesHandler(HandlerBase):
    def __init__(self, alphas, **kw):
        HandlerBase.__init__(self, **kw)
        self.alphas = alphas
        self.num_segments = len(alphas)
        
    def create_artists(self, legend, orig_handle, xdescent, ydescent, width, height, fontsize, trans):
        rect_width = width / self.num_segments
        rects = []
        for i, alpha in enumerate(self.alphas):
            rect = plt.Rectangle(
                (xdescent + i * rect_width, ydescent), 
                rect_width, 
                height, 
                facecolor=orig_handle.get_facecolor(), 
                alpha=alpha,
                edgecolor='black', 
                linewidth=0.3,
                transform=trans
            )
            rects.append(rect)
        return rects


In [41]:
#Just running the below so that it saves without plotting it.

In [43]:
from matplotlib import font_manager

def plot(time):
    gdata = merged[merged['TIME_PERIOD'] == time].copy()

    # Define colors and alphas
    color_map = {
        'China': '#FF0000',  # Red
        'United States': '#0000FF',  # Blue
        'EU (Member States and Institutions of the European Union) changing composition': '#FFD700'  # Gold
    }
    alpha_map = {1: 0.3, 2: 0.5, 3: 0.7, 4: 1.0}
    
    # Assign colors and alphas
    gdata['color'] = gdata['Highest_Exporter'].map(color_map)
    gdata['alpha'] = gdata['bin'].map(alpha_map)

    # Create figure with Robinson projection and center shifted left
    fig = plt.figure(figsize=(14, 8))
    # Shift center longitude to include more of Russia (standard is 0, moving to -10 or -20 will shift view east)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson(central_longitude=10))

    # Add coastlines but not borders
    ax.coastlines(linewidth=0.3)
    
    # Plot each combination of exporter and bin
    # We need to modify the plotting approach for GeoAxes
    for exporter, color in color_map.items():
        for bin_num, alpha in alpha_map.items():
            subset = gdata[(gdata['Highest_Exporter'] == exporter) & (gdata['bin'] == bin_num)]
            if not subset.empty:
                # Assuming you have geometry column in your data
                # If using GeoPandas:
                subset.plot(ax=ax, color=color, alpha=alpha, 
                           edgecolor='none',  # Hide borders between polygons
                           transform=ccrs.PlateCarree())

    # Custom legend with exporters appearing once and touching patches
    exporter_info = {
        'EU': '#FFD700',
        'China': '#FF0000',
        'USA': '#0000FF'
    }
    alphas = [0.2, 0.5, 0.7, 1.0]
    
    legend_handles = []
    legend_labels = []
    handler_map = {}
    
    for name, color in exporter_info.items():
        # Create single patch per exporter (the alpha will be handled by custom handler)
        handle = Patch(facecolor=color, edgecolor='black')
        legend_handles.append(handle)
        legend_labels.append(name)
        
        # Add custom handler for this patch
        handler_map[handle] = TouchingRectanglesHandler(alphas)

    # Create custom legend with wider boxes
    ax.legend(handles=legend_handles,
              labels=legend_labels,
              handler_map=handler_map,
              title="WHO IS THE LARGER TRADING PARTNER? (Percentage Share of the Three)",
              title_fontproperties= font_manager.FontProperties(weight='bold'),
              ncol=3,
              loc='upper center',
              bbox_to_anchor=(0.5, -0.05),
              frameon=False,
              handlelength=8)  # Much wider legend boxes

    # Title and cleanup
    ax.set_title(f"Top Import Source by Country ({time})", fontsize=14, fontweight= 'bold' ,pad=20)
    ax.axis('off')

    plt.tight_layout()
    plt.savefig(f'imports_app/plots/{time}.png',bbox_inches='tight', dpi=300) 
    plt.close()

import os

for year in merged['TIME_PERIOD'].dropna().unique():
    file_path = f'imports_app/plots/{year}.png'
    if not os.path.exists(file_path):
        plot(year)
    else:
        continue


